# Introduction to DataCamp Projects

## What are Projects?

In this project, we will analyze a fascinating dataset on every single lego block that has ever been built! Let us start by reading in the data, which has been downloaded from [Rebrickable](https://rebrickable.com/downloads/).

![lego](https://sh-s7-live-s.legocdn.com/is/image/LEGOMKTG/pick-a-brick--201606--gl--banner-background--small)

As a datacamp user, you have been working hard, learning the fundamentals of data science, by doing many [courses]() we offer, and practicing these skills by taking our [challenges](). 
Our vision behind __Projects__ is to take this many steps forward, and  allow you apply these skills to carry out an __end-to-end analysis__, on a __real-world task__, using __real-world__ tools and workflows, and be able to __showcase__ your work. In this __Introduction to Projects__, you will learn how projects work, and familiarize yourself with the new interface.

## The Projects Interface

The project interface consists of a __jupyter notebook__ on the right and an __instructional sidebar__ on the left.

1. The  [Jupyter Notebook](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html) on the right contains the main narrative for the project. It contains the story you will be working on, supported by all the code you write, and the output tables and plots, that you will create.

2. The __instructional sidebar__ on the left contains `instructions`, `hints`, and other pedagogical elements that will aid you in you in your quest to complete your project successfully and come up with a newsworthy story!

---

For your first task, all you have to do is to shift your attention to the content that is right on top of the notebook. Read the first paragraph to understand the narrative, press `Shift + Enter` twice, and notice what happens! If you are the curious type, please feel free to poke around the toolbar and mess with any and every element of the notebook. When you are done, click on `Check Project`. We will be waiting for you in the next task!

## Welcome to Jupyter!

Congratulations on completing your task successfully! How did it feel seeing the circle go green? Pretty awesome right?

![success](https://media.giphy.com/media/a0h7sAqON67nO/giphy.gif)

If this is your first brush with Jupyter Notebooks, continue reading this. Otherwise, go and dig into the notebook directly to explore lego blocks, and build your narrative! For those of you who choose to leave us here, don't worry, we will meet you again at the end of this journey.

As you might have already noticed, doing data science in the real world is all about building an interesting story around your data. As a result, your narrative around the data is as important as the code you write, and the outputs you generate. Jupyter Notebooks provide an awesome environment that allows you to do precisely this by combining text, code and outputs into one single document.

![jupyter-notebook](https://www.dataquest.io/blog/images/jupyter/interface-screenshot.png)

## Coding in Notebooks

A Jupyter Notebook consists of cells, each of which can contain code or markdown text. Jupyter Notebooks support code in multiple languages (like R, Python, Julia etc.), using a system of kernels. This notebook connects to a Python 3 kernel, and hence allows us to write Python code!  In this task, we will explore how to add a new cell to the notebook, write some python code to support the narrative, and execute it. Are you ready?

A picture is worth a thousand words. So instead of describing how to work with cells in a notebook, here is an interactive gif to walk you through the steps.

![Working with Cells](http://media.giphy.com/media/l378dk9Vx9JWj47ug/giphy.gif)

<hr/>

In order to complete this task, you need to do two things:

1. Add the following code to the active code cell, where it says `## YOUR CODE HERE`, and press `Shift + Enter` to execute it.

2. Insert a new cell by clicking on `Cell > Insert Cell Above`, change its type to `markdown`, enter some text (in markdown) that says something interesting about the output table just above it.

After you are completed this task, feel free to play around adding new cells, editing them, modifying their type, executing code, etc. Explore as much as you want to, and when you are ready click on `Next Task` to move on to the next task. Note that this task will NOT be checked at this point, and you will learn more about this in your next task.

## Working on Tasks

Every project is broken down into smaller tasks. In order to complete a task, you will need to:
    
1. Read the narrative in the notebook to understand the scope of the task.
2. Read the instructions in the sidebar to get more details on what is expected.
3. Write code in the notebook and run it by by pressing Shift + Enter, or clicking on the Run icon in the toolbar. 


## Checking your Work

At any point in the project, you can click on the __Check Project__ button at the bottom to test whether your output matches the solution.

- If all the tests pass, your task circle will turn green. 
- If some tests fail, your task circle with turn orange. 

You can view the test results in the sidebar to understand what failed and update your code accordingly.

## Getting Help

If you are unable to complete the task after repeated attempts, you can use the sidebar to ask for a hint. 

## Next Steps